In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
import matplotlib.pyplot as plt
def plot(x,y):
    plt.scatter(x, y, alpha=0.5)
    plt.show()
    return
def encoding(df, dictionary, feature):
    encoded_df = df.copy()
    for key in dictionary.keys():
        encoded_df.loc[encoded_df[feature] == key, feature] = dictionary[key]
    return encoded_df

In [23]:
training_df = pd.read_csv("train_Mona_dummied.csv")
training_df.head()

,LotFrontage,LotArea,MasVnrArea,BsmtUnfSF,TotalBsmtSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,...,YearRemodAdd_label_6,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,65.0,8450,196,150,856,1710,1,0,2,1,...,0,0,0,1,0,0,0,1,0,0
1,80.0,9600,0,284,1262,1262,0,1,2,0,...,0,0,0,0,1,0,1,0,0,0
2,68.0,11250,162,434,920,1786,1,0,2,1,...,0,0,0,1,0,0,0,1,0,0
3,60.0,9550,0,540,756,1717,1,0,1,0,...,0,0,0,1,0,1,0,0,0,0
4,84.0,14260,350,490,1145,2198,1,0,2,1,...,0,0,0,1,0,0,0,1,0,0


In [24]:
training_df.columns[training_df.isna().any()].tolist()

[]

In [38]:
X = training_df.drop(columns = ["target", "SalePrice"]).to_numpy()

In [39]:
y = training_df["target"]
y_actual = training_df["SalePrice"]

In [27]:
X.shape

(1455, 246)

In [28]:
y.shape

(1455,)

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    regr = LinearRegression()
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    y_train_pred = regr.predict(X_train)
    mape = np.mean(np.abs(y_pred - y_test)/y_test)
    mape_train = np.mean(np.abs(y_train_pred - y_train)/y_train)
    
    
    y_trans_pred = regr.predict(X)
    y_act_pred = np.exp(y_trans_pred) - 1
    mape_actual = np.mean(np.abs(y_act_pred - y_actual)/y_actual)
    
    print(mape, mape_actual)

0.007559687512496814 0.07188490686370219
0.007270523402228068 0.07118096954945606
0.006894642043840114 0.07100074957669253
0.007478517286861603 0.07134443911618729
0.007044992937210595 0.07036703277108103


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    regr = xgb.XGBRegressor(objective ='reg:linear')
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    y_train_pred = regr.predict(X_train)
    mape = np.mean(np.abs(y_pred - y_test)/y_test)
    mape_train = np.mean(np.abs(y_train_pred - y_train)/y_train)
    
    
    y_trans_pred = regr.predict(X)
    y_act_pred = np.exp(y_trans_pred) - 1
    mape_actual = np.mean(np.abs(y_act_pred - y_actual)/y_actual)
    
    print(mape, mape_actual)

/Applications/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:32:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.008295671807171867 0.07447017290201552
[21:32:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.00802593552405251 0.07553383851387438
[21:32:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.008275534601182077 0.0745526492832391
[21:32:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.007950390636973533 0.07478556327401818
[21:32:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.008207048113132904 0.07344843388007509


In [40]:
from sklearn.neural_network import MLPRegressor
for i in range(5):
    X_train, X_test, y_trans_train, y_trans_test = train_test_split(X, y, test_size=0.2)
    #linear = LinearRegression()
    linear = xgb.XGBRegressor(objective ='reg:linear')
    linear.fit(X_train[:,:16], y_trans_train)
    residual = y_trans_train - linear.predict(X_train[:,:16])
    #plot(y_train, linear.predict(X_train[:,:7]))
    rf = RandomForestRegressor(criterion = "mse", n_estimators = 200)
    #rf = MLPRegressor(hidden_layer_sizes = (20,20,20), max_iter = 2000, learning_rate = 'adaptive', n_iter_no_change = 100, verbose = True)
    rf.fit(X_train,residual)
    
    
    y_test_trans_pred = rf.predict(X_test) + linear.predict(X_test[:,:16])
    y_train_trans_pred = rf.predict(X_train) + linear.predict(X_train[:,:16])
    #plot(y_train, y_train_pred)
    #plot(y_test, y_test_pred)
    mape_test = np.mean(np.abs(y_test_trans_pred - y_trans_test)/y_trans_test)
    mape_train = np.mean(np.abs(y_train_trans_pred - y_trans_train)/y_trans_train)
    
    
    y_trans_pred = rf.predict(X) + linear.predict(X[:,:16])
    y_act_pred = np.exp(y_trans_pred) - 1
    mape_actual = np.mean(np.abs(y_act_pred - y_actual)/y_actual)
    print(mape_test, mape_train, mape_actual)

[21:38:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.00818523686358438 0.002458693299711983 0.04358740714440234
[21:38:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Applications/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.008437311577495266 0.002445554472745222 0.04425284280520848
[21:38:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Applications/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.008383703267155904 0.002434113731817784 0.04458175140881635
[21:38:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Applications/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.008098658832288661 0.002453719055212163 0.04331963026369525
[21:39:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Applications/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.007950227514321454 0.002520036262576435 0.04345329481072288


In [41]:
testing_df = pd.read_csv("test_final_dummied.csv")
testing_df.head()

,LotFrontage,LotArea,MasVnrArea,BsmtUnfSF,TotalBsmtSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,...,YearRemodAdd_label_6,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,80.0,11622,0,270,882,896,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
1,81.0,14267,108,406,1329,1329,0,0,1,1,...,1,0,0,1,0,0,0,0,0,1
2,74.0,13830,0,137,928,1629,0,0,2,1,...,0,0,0,0,1,0,0,0,0,1
3,78.0,9978,20,324,926,1604,0,0,2,1,...,0,0,0,1,0,0,0,0,0,1
4,43.0,5005,0,1017,1280,1280,0,0,2,0,...,0,0,0,1,0,0,0,0,0,1


In [42]:
testing_df.columns[testing_df.isna().any()].tolist()

[]

In [43]:
X_test = testing_df.drop(columns = ["target", "SalePrice"]).to_numpy()

In [44]:
y_test = testing_df["target"]
y_test_actual = testing_df["SalePrice"]

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
#regr = LinearRegression()
regr = xgb.XGBRegressor(objective ='reg:linear')
regr.fit(X, y)
y_pred = regr.predict(X_test)
y_train_pred = regr.predict(X)
mape = np.mean(np.abs(y_pred - y_test)/y_test)
mape_train = np.mean(np.abs(y_train_pred - y)/y)


y_trans_pred = regr.predict(X_test)
y_act_pred = np.exp(y_trans_pred) - 1
mape_actual = np.mean(np.abs(y_act_pred - y_test_actual)/y_test_actual)
print(mape, mape_actual)

/Applications/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Applications/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[21:40:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.007864756078093794 0.09984166155825819


In [49]:
from sklearn.neural_network import MLPRegressor

X_train = X
y_trans_train = y
best_mape = 1000000000
for i in range(5):
    linear = LinearRegression()
    #linear = xgb.XGBRegressor(objective ='reg:linear')
    linear.fit(X_train[:,:16], y_trans_train)
    residual = y_trans_train - linear.predict(X_train[:,:16])
    rf = RandomForestRegressor(criterion = "mse", n_estimators = 200)
    #rf = MLPRegressor(hidden_layer_sizes = (20,20,20), max_iter = 200000, activation = "relu", learning_rate = 'adaptive', n_iter_no_change = 1000, verbose = False)
    rf.fit(X_train,residual)
    plot(linear.predict(X_train[:,14:]), y_trans_train)
    plot(rf.predict(X_train), residual)
    
    y_trans_train_pred = rf.predict(X_train) + linear.predict(X_train[:,:16])
    mape_train_trans = np.mean(np.abs(y_trans_train_pred - y_trans_train)/y_trans_train)
    
    y_trans_pred = rf.predict(X_test) + linear.predict(X_test[:,:16])
    y_act_pred = np.exp(y_trans_pred) - 1
    mape_actual = np.mean(np.abs(y_act_pred - y_test_actual)/y_test_actual)
    #plot(y_test, y_trans_pred)
    print(mape_train_trans, mape_actual)
    
    if mape_actual <= best_mape:
        best_model_linear_part = linear
        best_model_rf_part = rf
        best_y_prediction = y_act_pred

NameError: name 'plot' is not defined